In [1]:
#インポート(ないならpip installでインストール)
import fitz          #PDF読み込むライブラリ(pip install PyMuPDF)
import pandas as pd  #表を生成するライブラリ(pip install pandas)
import numpy as np   #配列を便利に使えるようになるライブラリ(pip install numpy)
import os            #ファイルとディレクトリ操作するようのライブラリ
import glob
from pathlib import Path
import re
import datetime
import sys

In [3]:
#全角数字を半角数字にする用のテーブル
zenkaku_table = str.maketrans({
    '１': '1',
    '２': '2',
    '３': '3',
    '４': '4',
    '５': '5',
    '６': '6',
    '７': '7',
    '８': '8',
    '９': '9',
    '０': '0',
})

In [7]:
#ファイルのパスを取得
def make_file_path_list(dir):
    file_path_list = glob.glob(f'./{dir}/*')
    file_path_list.sort()
    
    return file_path_list

In [ ]:
def get_pdf_name:

In [9]:
def get_sale(PDF_name):
    try:
        doc = fitz.open(PDF_name)
        
        #売上高があるページの取得
        array = np.array(doc.get_toc())
        page_sales = array[np.any((array == "１主要な経営指標等の推移") | (array == "1主要な経営指標等の推移"), axis=1), :]
        page_int = int(page_sales[0, 2])
        page = doc[page_int - 1]
        
        #社名の取得
        text = doc.get_page_text(0)
        lines = text.splitlines()
        next_lines = []
        target_string = "会社名"
        for i in range(len(lines) - 1):
            if target_string in lines[i]:
                next_lines.append(lines[i + 1].strip())
                company_name = next_lines[0]
        
        #2ページ目にある売上高を含めた表の抽出
        tables = page.find_tables()
        
        #抽出した表から売上高を抽出する
        table_data = np.array(tables[0].extract())
        columns = table_data[0]
        a = table_data[np.any(table_data == "決算年月", axis=1), :]
        b = np.ravel(a)
        
        #全角を半角にする
        for i in range(b.size):
            word = b[i]
            b[i] = b[i].translate(zenkaku_table)
            numbers = re.findall(r'\d+', b[i])
            
            #和暦西暦変換
            characters = "昭平令"
            pattern = f"[{re.escape(characters)}]"
            matches = re.findall(pattern, b[i])
            
            #規格をそろえる
            if len(numbers) != 0:
                year = numbers[0]
                date = numbers[1]
                if len(matches) != 0:
                    if matches[0] == "令":
                        year = to_seireki('令', year)
                    elif matches[0] == "平":
                        year = to_seireki('平', year)
                    elif matches[0] == "昭":
                        year = to_seireki('昭', year)
                    else:
                        print("違う")
                b[i] = year + " - " + date
        
        data_rows = table_data[1:]
        
        #データの形状と列名の整合性を確認
        if len(data_rows) > 0 and len(data_rows[0]) != len(b):
            raise ValueError("データの列数が一致しない")
        
        df = pd.DataFrame(data_rows, columns=b)
        sales = df[df['決算年月'].str.contains('売上', na=False)]
        if not sales.empty:
            sales.iat[0, 0] = company_name
        else:
            print(f"{company_name}: 売上データ。")
        
        return sales
    except Exception as e:
        print(f"{PDF_name} の処理中にエラー発生: {e}")
        return pd.DataFrame()  # 空のDataFrameを返す

In [5]:
#売上高だけのデータフレームを作成する関数
def make_dataframe(sales):
    a = pd.DataFrame()
    df = a.append(sales, ignore_index = True)
    return df

In [6]:
def to_seireki(era, year):
    ERA_DICT = {'令': datetime.datetime(2019, 5, 1),'平': datetime.datetime(1989, 1, 8),'昭': datetime.datetime(1926, 12, 25)}
    era_start = ERA_DICT[era]
    year_1 = int(year)
    jp_year = era_start.year + year_1 - 1
    return str(jp_year)

In [13]:
def main(dir):
    df = pd.DataFrame()
    file_path_list = make_file_path_list(dir)
    #for i in range(58,59):
    for i in range(1,len(file_path_list)):
        pdf_name = file_path_list[i]
        sales = get_sale(pdf_name)
        df = pd.concat([df,sales])
    return df

In [12]:
df = main('yuho_report')

<class 'numpy.ndarray'>


UnboundLocalError: cannot access local variable 'data_rows' where it is not associated with a value

In [1]:
file_path_list = make_file_path_list('yuho_report')

NameError: name 'make_file_path_list' is not defined